Для работы нашего генератора нам требуется понимание с каких ip-адресов на какие комьютеры (какие ip-адреса), 
с каких портов и на какие порты отправляются сетевые пакеты с данными как при атаках, так и при обычных 
пользовательских запросах. Это необходимо знать для того, чтобы сгенерировать пакеты с данными, которые 
будут приближены к данным в том датасете, на котором обучается модель машинного обучения и 
нейросеть для детектирования атак.

Этот файл посвящен тому, чтобы вычленить из нашего ip-адреса машин с которых отправляются данные, 
сгруппировать их по признаку боты это или обычные пользовательские комьютеры. И для каждой такой машины
агрегировать список ip-адресов на которые она отправляла запросы, список портов с которых отправлялись
сетевые пакеты, список портов на которые отправлялись пакеты, среднее время отправки пакетов данных,
средняя длина пакетов, среднее количество пакетов во время одной сессии и т.п. Эти данные позволят нам
генерировать похожие пакеты для найденных ip-адресов комьютеров

Для примера был взят файл с https://www.kaggle.com/devendra416/ddos-datasets. Был взят датасет из папки
ddos_balanced. Т.к. размер файла с данными очень большой (более 6 ГБ). Приходится применять специальные механизмы
для загрузки данных. Для этого:
1. Файл загружался блоками (chunks) по 400 тыс. строк в каждом блоке (значение переменной size)
2. При загружке данных они сразу сжимались, путем группировки данных по ip-адресу машины источника данных 
(Поле Src IP), вычислялись средние числовые показатели, а так же группировались во множества (set) нечисловые
(категориальные) параметры (порт источника, порт назначения, ip-адрес назначения и др.). Т.к. в python множество - 
это структура данных, которая содержит только уникальные элементы, то при группировке данных значений во множество,
остаются только уникальные значения этих параметров в рамках данного блока (chunk) данных из большого файла, т.е.
для одного ip-адреса в поле Dst IP будут храниться только уникальные ip-адреса, для Protocol - только уникальные
номера протоколов по которым отправлял данные этот комьютер и т.д. Это значительно сокращает объем получаемых данных

In [1]:
import pandas as pd
import numpy as np
size = 4 * 10 ** 4
readerCSV = pd.read_csv("/Users/evgenii/DDoS Dataset/final_dataset.csv", sep=",", chunksize=size)

ddos_result = []
i = 0
iter_count = 12794627 / size

for data in readerCSV:
    
    data = data.fillna(0)


    flows_count = data.groupby(['Src IP', 'Dst IP']).agg({'Flow ID': 'count'})

    mean_flows_count = flows_count.groupby(['Src IP']).mean().rename(columns={'Flow ID': 'Flows Cnt Mean'})

    std_flows_count = flows_count.groupby(['Src IP']).std().rename(columns={'Flow ID': 'Flows Cnt Std'})

    cnt_data = data.groupby(['Src IP']).mean()[['Tot Fwd Pkts', 'Tot Bwd Pkts', 'Fwd Pkt Len Std', 
                                                'Fwd Pkt Len Mean', 'Bwd Pkt Len Std', 'Bwd Pkt Len Mean', 'Flow IAT Mean', 
                                                'Flow IAT Std', 'Fwd PSH Flags', 'Bwd PSH Flags', 'SYN Flag Cnt', 
                                                'FIN Flag Cnt', 'URG Flag Cnt', 'Flow Byts/s', 'Flow Pkts/s', 'Bwd Pkts/s', 
                                                'Down/Up Ratio', 'Idle Std', 'Idle Mean', 'Active Mean', 'Active Std']]

    dst_ip_data = data.groupby(['Src IP']).agg({'Dst IP': set})
    protocol_data = data.groupby(['Src IP']).agg({'Protocol': set})
    dst_port_data = data.groupby(['Src IP']).agg({'Dst Port': set})
    src_ports_data = data.groupby(['Src IP']).agg({'Src Port': set})
    labels_data = data.groupby(['Src IP']).agg({'Label': set})
    duration_data = data.groupby(['Src IP']).agg({'Flow Duration': list})

    ddos_data = src_ports_data.join(dst_ip_data).join(dst_port_data).join(protocol_data) \
        .join(labels_data).join(duration_data).join(cnt_data).join(mean_flows_count).join(std_flows_count)

    ddos_result.append(ddos_data)

    i += 1
    percent = i / iter_count * 100
    print("Чтение данных: " + "{0:.2f}".format(percent if percent < 100 else 100) + "%", end="\r")

print()
print('finished')
readerCSV = None

data[['Flow Duration', 'Flow Pkts/s', 'Tot Fwd Pkts', 'Tot Bwd Pkts', 'Bwd Pkts/s']]

Чтение данных: 100.00%
finished


,Flow Duration,Flow Pkts/s,Tot Fwd Pkts,Tot Bwd Pkts,Bwd Pkts/s
12760000,1515,1.320132e+03,1,1,660.066007
12760001,70,2.857143e+04,2,0,0.000000
12760002,60716590,4.446890e-01,14,13,0.214110
12760003,1,2.000000e+06,2,0,0.000000
12760004,5029754,4.771605e+00,10,14,2.783436
...,...,...,...,...,...
12794622,50,4.000000e+04,1,1,20000.000000
12794623,306,9.803922e+03,3,0,0.000000
12794624,362,5.524862e+03,1,1,2762.430939
12794625,303,6.600660e+03,1,1,3300.330033


In [2]:
intermediate_res = pd.concat(ddos_result, sort=False)
ddos_result = []
intermediate_res

,Src Port,Dst IP,Dst Port,Protocol,Label,Flow Duration,Tot Fwd Pkts,Tot Bwd Pkts,Fwd Pkt Len Std,Fwd Pkt Len Mean,...,Flow Byts/s,Flow Pkts/s,Bwd Pkts/s,Down/Up Ratio,Idle Std,Idle Mean,Active Mean,Active Std,Flows Cnt Mean,Flows Cnt Std
Src IP,,,,,,,,,,,,,,,,,,,,,
192.168.1.101,"{4100, 2055, 2057, 2061, 2068, 2069, 2076, 208...","{97.74.104.201, 97.74.144.108, 74.55.1.4}","{80, 443}",{6},{ddos},"[6163053, 95002467, 1421816, 5687268, 5225668,...",6.884336,8.625909,31.156098,24.178063,...,1.135377e+03,2.606293e+02,127.428095,0.571051,5.361768e+04,6.644611e+06,8.336376e+04,35300.062284,504.333333,844.953450
192.168.1.102,"{3076, 3077, 3079, 3082, 3083, 3088, 3090, 309...","{97.74.104.201, 69.84.133.138, 74.55.1.4}",{80},{6},{ddos},"[119942533, 14722879, 119777479, 111202320, 10...",84.918768,159.750700,15.526194,6.713555,...,1.684375e+04,4.281858e+04,21270.000309,0.980392,3.659573e+04,1.003268e+06,3.734518e+05,9916.368445,119.000000,171.717792
192.168.1.103,"{2049, 2563, 2052, 2053, 2054, 3075, 3258, 358...","{69.192.24.88, 97.74.144.108, 208.113.162.153,...",{80},{6},{ddos},"[4134203, 5977626, 3040957, 632174, 3439332, 4...",38.111709,62.118439,18.355966,9.952544,...,2.258371e+05,9.638300e+03,4802.931374,0.768506,3.716657e+04,1.711862e+06,5.116172e+05,66589.236295,148.600000,224.602538
192.168.1.104,"{21574, 18449, 18450, 18451, 20523, 20524, 205...","{203.73.24.75, 97.74.104.201, 67.220.214.50, 9...","{80, 443}",{6},{ddos},"[6108487, 95010697, 1421818, 5577930, 216986, ...",5.443097,9.551819,30.109324,19.114062,...,2.976592e+03,3.943954e+04,19220.034793,1.319124,0.000000e+00,3.850293e+05,1.631446e+04,0.000000,426.166667,846.936460
192.168.1.105,"{17408, 18560, 17412, 17418, 18582, 18585, 184...",{97.74.104.201},"{80, 443}",{6},{ddos},"[79516370, 119531746, 113020493, 119974637, 89...",71.464286,124.642857,21.366217,12.969670,...,3.933222e+03,1.673722e+02,87.318461,0.583333,1.569676e+06,7.704307e+06,3.369959e+06,892907.549405,84.000000,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96.6.49.94,{443},"{172.31.65.65, 172.31.65.113, 172.31.64.69}","{51987, 49972, 50030}",{6},{Benign},"[5184, 54, 434]",3.000000,1.000000,17.897858,10.333333,...,2.171609e+05,2.802076e+04,7005.189073,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,1.000000,0.000000
96.6.55.102,{443},{172.31.69.4},{49802},{6},{Benign},[1],2.000000,0.000000,21.920310,15.500000,...,3.100000e+07,2.000000e+06,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,1.000000,NaN
97.68.80.146,{37767},{172.31.65.76},{445},{6},{Benign},[48144],3.000000,1.000000,0.000000,0.000000,...,0.000000e+00,8.308408e+01,20.771020,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,1.000000,NaN


В связи с тем, что мы сгруппировали только данные внутри блоков и оставили уникальные ip-адреса источников данных только
внутри каждого блока (chunk) из файла - тем не менее между блоками ip-адреса так же могут пересекаться. Необходимо
теперь провести аналогичную группировку данных по всей собранной таблице, попутно вычислив средние показатели для
непрерывных переменных, и собрать получившиеся множества категориальных переменных в единый блок для каждого ip-адреса.
Т.е. из каждого блока (chunk) для конкретного ip-адреса собрать вместе все ip-адреса назначения, протоколы, порты источника,
порты назначения и т.п. Но сделать это простым методом не удастся. Т.к. в каждом блоке (chunk) данные были собраны во
множества - нет возмоности средствами pandas объединить множества при очередной агрегации (во всяком случае я его не
нашел). Поэтому мы объединяем множества в списки (получается список множеств) и позже его обработаем для большего удобства


In [3]:
# вновь агрегируем данные по ip-адресу источника уже по единой таблице, вычисляя средние значения числовых параметров
cnt_data = intermediate_res.groupby(['Src IP']).mean()[['Tot Fwd Pkts', 'Tot Bwd Pkts', 'Fwd Pkt Len Std', 
                                                'Fwd Pkt Len Mean', 'Bwd Pkt Len Std', 'Bwd Pkt Len Mean', 'Flow IAT Mean', 
                                                'Flow IAT Std', 'Fwd PSH Flags', 'Bwd PSH Flags', 'SYN Flag Cnt', 
                                                'FIN Flag Cnt', 'URG Flag Cnt', 'Flow Byts/s', 'Flow Pkts/s', 'Bwd Pkts/s', 
                                                'Down/Up Ratio', 'Idle Std', 'Idle Mean', 'Active Mean', 'Active Std',
                                                'Flows Cnt Mean', 'Flows Cnt Std']]

# вновь агрегируем данные по ip-адресу источника уже по единой таблице, объединяя множества категориальных переменных
# в списки множеств (как описано выше)
dst_ip_data = intermediate_res.groupby(['Src IP'])['Dst IP'].apply(np.hstack).to_frame()
protocol_data = intermediate_res.groupby(['Src IP'])['Protocol'].apply(np.hstack).to_frame()
dst_port_data = intermediate_res.groupby(['Src IP'])['Dst Port'].apply(np.hstack).to_frame()
src_ports_data = intermediate_res.groupby(['Src IP'])['Src Port'].apply(np.hstack).to_frame()
labels_data = intermediate_res.groupby(['Src IP'])['Label'].apply(np.hstack).to_frame()
duration_data = intermediate_res.groupby(['Src IP'])['Flow Duration'].apply(np.hstack).to_frame()

result_data = dst_ip_data.join(protocol_data).join(dst_port_data).join(src_ports_data).join(duration_data).join(labels_data).join(cnt_data)
result_data

,Dst IP,Protocol,Dst Port,Src Port,Flow Duration,Label,Tot Fwd Pkts,Tot Bwd Pkts,Fwd Pkt Len Std,Fwd Pkt Len Mean,...,Flow Byts/s,Flow Pkts/s,Bwd Pkts/s,Down/Up Ratio,Idle Std,Idle Mean,Active Mean,Active Std,Flows Cnt Mean,Flows Cnt Std
Src IP,,,,,,,,,,,,,,,,,,,,,
1.0.132.203,[{172.31.64.91}],[{17}],[{38840}],[{60449}],[20026],[{Benign}],2.000000,0.000000,235.466558,196.500000,...,19624.488165,99.870169,0.000000,0.0,0.000000e+00,0.0,0.0,0.0,1.0,NaN
1.0.176.98,"[{172.31.67.104}, {172.31.67.48}, {172.31.64.4...","[{6}, {6}, {6}, {6}, {6}, {6}, {6}, {6}, {6}, ...","[{445}, {445}, {445}, {445}, {445}, {445}, {44...","[{15281}, {50483}, {52319}, {30317}, {48026}, ...","[257492, 254493, 770967, 797106, 260423, 80654...","[{Benign}, {Benign}, {Benign}, {Benign}, {Beni...",4.602273,3.079545,37.279677,40.646591,...,417.149021,12.479065,4.373826,0.0,0.000000e+00,0.0,0.0,0.0,1.0,0.0
1.1.196.153,"[{172.31.64.99}, {172.31.65.48}, {172.31.65.48...","[{6}, {6}, {6}, {6}]","[{445}, {445}, {445}, {445}]","[{63015}, {56757}, {58357}, {63057}]","[263961, 266850, 809083, 1344010]","[{Benign}, {Benign}, {Benign}, {Benign}]",4.500000,2.750000,29.875229,31.650000,...,344.366475,12.548914,4.049985,0.0,0.000000e+00,0.0,0.0,0.0,1.0,NaN
1.1.207.23,"[{172.31.67.19}, {172.31.67.19}, {172.31.64.14...","[{6}, {6}, {6}, {6}, {6}, {6}, {6}]","[{445}, {445}, {445}, {445}, {445}, {445}, {445}]","[{63898}, {63869}, {56816}, {56785}, {54095}, ...","[870160, 282973, 807057, 266999, 360961, 27492...","[{Benign}, {Benign}, {Benign}, {Benign}, {Beni...",3.857143,2.285714,25.036312,27.000000,...,246.741373,12.397241,3.989102,0.0,0.000000e+00,0.0,0.0,0.0,1.0,NaN
1.1.223.186,"[{172.31.67.62}, {172.31.67.62}, {172.31.67.62}]","[{6}, {6}, {6}]","[{445}, {445}, {445}]","[{26367}, {26340}, {26290}]","[834089, 271458, 838027]","[{Benign}, {Benign}, {Benign}]",4.333333,3.000000,40.922982,43.933333,...,399.013763,12.088324,4.417526,0.0,0.000000e+00,0.0,0.0,0.0,1.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99.229.225.189,"[{172.31.65.25}, {172.31.65.25}]","[{6}, {6}]","[{30303}, {30303}]","[{54058}, {52240}]","[2993115, 3005821]","[{Benign}, {Benign}]",2.000000,0.000000,0.000000,0.000000,...,0.000000,0.666788,0.000000,0.0,0.000000e+00,0.0,0.0,0.0,1.0,NaN
99.230.229.253,[{172.31.64.109}],[{6}],[{30303}],[{64634}],[2160461],[{Benign}],3.000000,0.000000,0.000000,0.000000,...,0.000000,1.388593,0.000000,0.0,0.000000e+00,0.0,0.0,0.0,1.0,NaN
99.26.255.249,[{172.31.64.113}],[{6}],[{3389}],[{54288}],[76657714],[{Benign}],8.000000,17.000000,195.839402,130.250000,...,36.630364,0.326125,0.221765,2.0,1.245841e+07,19046115.0,19519369.0,0.0,1.0,NaN


In [4]:
for index, row in result_data.iterrows():
    p = set()
    for d in row['Dst IP']:
        p |= d
    result_data.loc[index,'Dst IP'] = " ".join(map(str, p))

    p = set()
    for d in row['Protocol']:
        p |= d
    result_data.loc[index,'Protocol'] = " ".join(map(str, p))
    
    p = set()
    for d in row['Dst Port']:
        p |= d
    result_data.loc[index,'Dst Port'] = " ".join(map(str, p))
    
    p = set()
    for d in row['Src Port']:
        p |= d
    result_data.loc[index,'Src Port'] = " ".join(map(str, p))
    
    p = set()
    for d in row['Label']:
        p |= d
    result_data.loc[index,'Label'] = " ".join(map(str, p))

    result_data.loc[index, 'Flow Duration Mean'] = np.mean(row['Flow Duration'].tolist())
    result_data.loc[index, 'Flow Duration Std'] = np.std(row['Flow Duration'].tolist())

del result_data['Flow Duration']
result_data = result_data.fillna(0)
result_data

,Dst IP,Protocol,Dst Port,Src Port,Label,Tot Fwd Pkts,Tot Bwd Pkts,Fwd Pkt Len Std,Fwd Pkt Len Mean,Bwd Pkt Len Std,...,Bwd Pkts/s,Down/Up Ratio,Idle Std,Idle Mean,Active Mean,Active Std,Flows Cnt Mean,Flows Cnt Std,Flow Duration Mean,Flow Duration Std
Src IP,,,,,,,,,,,,,,,,,,,,,
1.0.132.203,172.31.64.91,17,38840,60449,Benign,2.000000,0.000000,235.466558,196.500000,0.000000,...,0.000000,0.0,0.000000e+00,0.0,0.0,0.0,1.0,0.0,2.002600e+04,0.000000
1.0.176.98,172.31.64.44 172.31.65.10 172.31.66.49 172.31....,6,445,52097 24706 31749 23054 19727 18706 13588 5838...,Benign,4.602273,3.079545,37.279677,40.646591,53.175738,...,4.373826,0.0,0.000000e+00,0.0,0.0,0.0,1.0,0.0,7.036811e+05,380577.391108
1.1.196.153,172.31.64.99 172.31.65.48,6,445,63057 58357 56757 63015,Benign,4.500000,2.750000,29.875229,31.650000,57.447203,...,4.049985,0.0,0.000000e+00,0.0,0.0,0.0,1.0,0.0,6.709760e+05,447500.654387
1.1.207.23,172.31.67.19 172.31.64.14,6,445,56816 56785 64357 57734 63898 63869 54095,Benign,3.857143,2.285714,25.036312,27.000000,26.621372,...,3.989102,0.0,0.000000e+00,0.0,0.0,0.0,1.0,0.0,5.310179e+05,272896.016904
1.1.223.186,172.31.67.62,6,445,26290 26340 26367,Benign,4.333333,3.000000,40.922982,43.933333,41.517181,...,4.417526,0.0,0.000000e+00,0.0,0.0,0.0,1.0,0.0,6.478580e+05,266159.847912
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99.229.225.189,172.31.65.25,6,30303,52240 54058,Benign,2.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.0,0.000000e+00,0.0,0.0,0.0,1.0,0.0,2.999468e+06,6353.000000
99.230.229.253,172.31.64.109,6,30303,64634,Benign,3.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.0,0.000000e+00,0.0,0.0,0.0,1.0,0.0,2.160461e+06,0.000000
99.26.255.249,172.31.64.113,6,3389,54288,Benign,8.000000,17.000000,195.839402,130.250000,271.869197,...,0.221765,2.0,1.245841e+07,19046115.0,19519369.0,0.0,1.0,0.0,7.665771e+07,0.000000


In [5]:
# отфильтруем строки таблицы по метке (ddos - источник отправлял пакеты для ddos атаки)
ddos_data = result_data[result_data['Label'] == 'ddos']
# запишем отфильтрованную таблицу в отдельный файлd dos_src.csv - можете его посмотреть, он не очень большой
ddos_data.to_csv("ddos_src.csv")
# посмотрим сколько же строк, т.е. ip-адресов в этой таблице. Оказывается, что ddos атаки в этом датасете производились
# всего с 34 комьютеров. 
len(ddos_data)

34

In [6]:
# отфильтруем строки таблицы по метке (Benign - источник отправлял пакеты для обычных пользвательских запросов)
benign_data = result_data[result_data['Label'] == 'Benign']

benign_data.to_csv("benign_src.csv")
# в этом подмножестве таблицы оказалось 36724 адреса. Можем использовать эти данные для генерации пакетов чистых
# пользовательских запросов
len(benign_data)

36724

In [7]:
ddos_data['Flow Pkts/s']

Src IP
13.59.126.31      7.334332e+05
18.216.200.189    6.971525e+02
18.216.24.42      6.508006e+02
18.217.165.70     6.529194e+04
18.218.11.51      6.422018e+02
18.218.115.60     6.236844e+02
18.218.229.235    6.408871e+02
18.218.55.126     6.142707e+02
18.219.193.20     1.172711e+03
18.219.211.138    1.069695e+01
18.219.32.43      6.830213e+02
18.219.5.43       7.365406e+02
18.219.9.1        6.510243e+02
192.168.1.101     1.305487e+02
192.168.1.102     2.143187e+04
192.168.1.103     4.821816e+03
192.168.1.104     1.972396e+04
192.168.1.105     8.764766e+01
192.168.2.108     8.315278e+02
192.168.2.109     8.001873e+03
192.168.2.110     1.503403e+04
192.168.2.111     5.723639e+00
192.168.2.112     3.497937e+03
192.168.2.113     9.997814e+01
192.168.3.114     1.325959e+04
192.168.3.115     3.644347e+02
192.168.3.117     1.612667e+03
192.168.4.118     2.380628e+04
192.168.4.119     1.577380e+03
192.168.4.120     2.601770e+04
192.168.4.121     1.640081e+00
192.168.56.1               inf
1